In [16]:
import numpy as np
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

In [17]:
df = pd.read_csv('../../../data/covid_toy.csv')

In [18]:
df.sample(5)

,age,gender,fever,cough,city,has_covid
56,71,Male,NaN,Strong,Kolkata,No
20,12,Male,98.0,Strong,Bangalore,No
85,16,Female,103.0,Mild,Bangalore,Yes
76,80,Male,100.0,Mild,Bangalore,Yes
90,59,Female,99.0,Strong,Delhi,No


In [19]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,0:5],df.iloc[:,-1],test_size=0.2,random_state=0)

# Without using Column Transformer

In [21]:
# adding simple imputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also test data
X_test_fever = si.transform(X_test[['fever']])

X_train_fever.shape

(80, 1)

In [22]:
# Ordinal Encoding --> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also for test data
X_test_cough = oe.transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [28]:
# One Hot Encoding --> gender,city
ohe = OneHotEncoder(drop='first',sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also for test data
X_test_gender_city = ohe.transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [33]:
# Extracting Age 
X_train_age = X_train.drop(columns=['gender','fever','city','cough']).values

# also test data
X_test_age = X_test.drop(columns=['gender','fever','city','cough']).values

X_train_age.shape

(80, 1)

In [38]:
# joining all the columns together 
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)

# also test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

# With Column Transformer

In [39]:
from sklearn.compose import ColumnTransformer

In [40]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),#--> a tuple with column transformer name, transformer object, column name
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city']),
],remainder='passthrough')# --> drop can also be used if the column wihtout a transformer is not needed . In this case the age column is needed therefore the parameter passthrough

In [44]:
X_train_transformed=transformer.fit_transform(X_train)
X_train_transformed.shape

(80, 7)

In [45]:
X_test_transformed = transformer.transform(X_test)
X_test_transformed.shape

(20, 7)